In [ ]:
# Import library yang diperlukan
from Bio import SeqIO
from Bio import Entrez
from Bio.Seq import Seq
from Bio.Blast import NCBIWWW
from Bio.Blast import NCBIXML
import collections
import re

# Notebook Analisis Genome Virus
# ==============================

# 1. Pengambilan Data Genome
# -------------------------

# Mengatur email untuk akses NCBI
Entrez.email = "ammarqurthuby@gmail.com"

# Mengambil data genome dari NCBI
def ambil_data_genome(accession_id):
    handle = Entrez.efetch(db="nucleotide", 
                          id=accession_id,  
                          rettype="gb", 
                          retmode="text")
    return SeqIO.read(handle, "genbank")

# Mengambil record untuk virus Barmah Forest
record = ambil_data_genome("NC_001786.1")


In [2]:
# 2. Analisis Komposisi Basa
# --------------------------

def hitung_komposisi_basa(sequence):
    """Menghitung jumlah setiap basa nukleotida dalam sequence"""
    return collections.Counter(str(sequence))

# Hitung komposisi basa untuk seluruh genome
komposisi_genome = hitung_komposisi_basa(record.seq)
print("\nKomposisi basa dalam genome lengkap:")
for basa, jumlah in komposisi_genome.items():
    print(f"Basa {basa}: {jumlah}")


Komposisi basa dalam genome lengkap:
Basa A: 3416
Basa T: 2494
Basa G: 2761
Basa C: 2815
Basa N: 2


In [3]:
# 3. Mengumpulkan dan Menganalisis CDS
# -----------------------------------

def gabung_cds(record):
    """Mengambil dan menggabungkan semua CDS dari genome"""
    cds_sequences = []
    for feature in record.features:
        if feature.type == "CDS":
            cds_sequences.append(str(feature.location.extract(record.seq)))
    return "".join(cds_sequences)

# Gabungkan semua CDS
gabungan_cds = gabung_cds(record)
komposisi_cds = hitung_komposisi_basa(gabungan_cds)

print("\nKomposisi basa dalam gabungan CDS:")
for basa, jumlah in komposisi_cds.items():
    print(f"Basa {basa}: {jumlah}")


Komposisi basa dalam gabungan CDS:
Basa A: 5501
Basa T: 3952
Basa G: 4591
Basa C: 4741
Basa N: 4


In [4]:
# 4. Mencari Karakter Aneh
# -----------------------

def cari_karakter_aneh(sequence):
    """Mencari karakter selain ATGC dalam sequence"""
    posisi_aneh = []
    for i, basa in enumerate(str(sequence)):
        if basa not in 'ATGC':
            posisi_aneh.append((i+1, basa))
    return posisi_aneh

karakter_aneh = cari_karakter_aneh(record.seq)
print("\nKarakter aneh yang ditemukan:")
for posisi, karakter in karakter_aneh:
    print(f"Posisi {posisi}: {karakter}")


Karakter aneh yang ditemukan:
Posisi 1857: N
Posisi 8101: N


In [5]:
# 5. Identifikasi Structural Polyprotein
# ------------------------------------

def cari_structural_polyprotein(record):
    """Mencari dan mengembalikan sequence structural polyprotein"""
    for feature in record.features:
        if feature.type == "CDS":
            if "product" in feature.qualifiers:
                if "structural polyprotein" in feature.qualifiers["product"][0].lower():
                    return feature.location.extract(record.seq)
    return None

polyprotein_seq = cari_structural_polyprotein(record)
if polyprotein_seq:
    print("\nUrutan Structural Polyprotein:")
    print(polyprotein_seq)


Urutan Structural Polyprotein:
ATGGCGAAACCAGTTGTGAAGATCGACGTGGAACCTGAAAGCCATTTCGCTAAGCAGGTCCAGAGTTGCTTCCCGCAGTTTGAGATCGAAGCAGTGCAGACCACACCAAACGATCATGCACACGCGAGGGCGTTTTCGCACCTTGCTACGAAGCTCATAGAAATGGAGACAGCAAAAGATCAGATCATCCTCGATATCGGAAGTGCACCCGCGAGGAGACTGTATTCAGAACACAAGTACCACTGTGTTTGCCCAATGAAGTGCACGGAAGATCCAGAGAGAATGCTAGGATATGCACGTAAGTTGATCGCAGGCTCTGCGAAAGGGAAGGCAGAAAAGTTACGCGATCTCAGGGATGTCTTGGCTACGCCAGACATCGAGACGCAGTCGCTATGTCTCCACACAGACGCATCCTGCAGATACCGCGGTGATGTTGCCGTGTATCAAGACGTGTATGCCATTGACGCACCTACCACGCTGTACCACCAAGCGTTAAAGGGCGTCAGGACCGCATATTGGATAGGCTTTGATACAACGCCGTTCATGTACGATGCACTAGCAGGAGCTTACCCGCTCTACTCCACAAACTGGGCTGATGAGCAAGTGCTCGAGTCCAGAAACATTGGGCTATGTTCAGACAAAGTTTCTGAAGGGGGAAAGAAAGGGAGATCAATCCTCAGGAAGAAGTTCTTGAAGCAGTCAGACAGAGTCATGTTCTCTGTCGGCTCGACGTTGTATACGGAAAGCCGTAAATTACTGCAAAGTTGGCACCTGCCATCCACATTCCATCTCAAAGGCAAATCTTCGTTCACGTGCCGCTGCGACACTATCGTCAGCTGCGAAGGGTATGTTCTGAAGAAAATTACAATGTGTCCTGGAGTGACAGGCAAACCGATAGGATATGCCGTCACCCATCACAAAGAAGGATTCGTAGTCGGAAAAGTCACAGATACCATTCGCGGCGAGAG

In [6]:
# Import library yang diperlukan
from Bio import SeqIO
from Bio.Seq import Seq
from Bio.SeqRecord import SeqRecord

def simpan_polyprotein_fasta(polyprotein_seq, nama_file):
    """
    Menyimpan sequence polyprotein dalam format FASTA
    
    Parameters:
    polyprotein_seq : Seq object atau string
        Sequence polyprotein yang akan disimpan
    nama_file : string
        Nama file output (termasuk ekstensi .fasta)
    """
    # Membuat SeqRecord object
    record = SeqRecord(
        Seq(str(polyprotein_seq)),
        id="BFV_structural_polyprotein",
        description="Barmah Forest Virus Structural Polyprotein"
    )
    
    # Menyimpan dalam format FASTA
    SeqIO.write(record, nama_file, "fasta")
    print(f"File telah disimpan sebagai {nama_file}")

# Contoh penggunaan:
if polyprotein_seq:  # memastikan polyprotein_seq sudah didefinisikan
    simpan_polyprotein_fasta(polyprotein_seq, "BFV_structural_polyprotein.fasta")
    print(polyprotein_seq)

File telah disimpan sebagai BFV_structural_polyprotein.fasta
ATGGCGAAACCAGTTGTGAAGATCGACGTGGAACCTGAAAGCCATTTCGCTAAGCAGGTCCAGAGTTGCTTCCCGCAGTTTGAGATCGAAGCAGTGCAGACCACACCAAACGATCATGCACACGCGAGGGCGTTTTCGCACCTTGCTACGAAGCTCATAGAAATGGAGACAGCAAAAGATCAGATCATCCTCGATATCGGAAGTGCACCCGCGAGGAGACTGTATTCAGAACACAAGTACCACTGTGTTTGCCCAATGAAGTGCACGGAAGATCCAGAGAGAATGCTAGGATATGCACGTAAGTTGATCGCAGGCTCTGCGAAAGGGAAGGCAGAAAAGTTACGCGATCTCAGGGATGTCTTGGCTACGCCAGACATCGAGACGCAGTCGCTATGTCTCCACACAGACGCATCCTGCAGATACCGCGGTGATGTTGCCGTGTATCAAGACGTGTATGCCATTGACGCACCTACCACGCTGTACCACCAAGCGTTAAAGGGCGTCAGGACCGCATATTGGATAGGCTTTGATACAACGCCGTTCATGTACGATGCACTAGCAGGAGCTTACCCGCTCTACTCCACAAACTGGGCTGATGAGCAAGTGCTCGAGTCCAGAAACATTGGGCTATGTTCAGACAAAGTTTCTGAAGGGGGAAAGAAAGGGAGATCAATCCTCAGGAAGAAGTTCTTGAAGCAGTCAGACAGAGTCATGTTCTCTGTCGGCTCGACGTTGTATACGGAAAGCCGTAAATTACTGCAAAGTTGGCACCTGCCATCCACATTCCATCTCAAAGGCAAATCTTCGTTCACGTGCCGCTGCGACACTATCGTCAGCTGCGAAGGGTATGTTCTGAAGAAAATTACAATGTGTCCTGGAGTGACAGGCAAACCGATAGGATATGCCGTCACCCATCACAAAGAAGGATTCGTAGTCGGA

In [18]:
# 6. BLAST untuk Mencari Virus Serupa
# ---------------------------------

# Import library yang diperlukan
from Bio import SeqIO
from Bio.Seq import Seq
from Bio.SeqRecord import SeqRecord

def translate_and_save_for_blast(nucleotide_seq, nama_file="protein_untuk_blast.fasta"):
    """
    Menerjemahkan sequence DNA menjadi protein dan menyimpan dalam format FASTA
    """
    # Menerjemahkan sequence DNA menjadi protein
    protein_seq = nucleotide_seq.translate()
    
    # Membuat SeqRecord object untuk protein
    record = SeqRecord(
        protein_seq,
        id="BFV_structural_polyprotein",
        description="Barmah Forest Virus Structural Polyprotein - Translated"
    )
    
    # Menyimpan dalam format FASTA
    SeqIO.write(record, nama_file, "fasta")
    print(f"File protein telah disimpan sebagai {nama_file}")
    print("\nLangkah BLAST:")
    print("1. Buka https://blast.ncbi.nlm.nih.gov/Blast.cgi")
    print("2. Pilih 'Protein BLAST (blastp)'")
    print("3. Upload file FASTA protein yang telah disimpan")
    print("4. Atur parameter BLAST:")
    print("   - Database: nr (non-redundant protein sequences)")
    print("   - Max target sequences: 10")
    print("   - Expect threshold: 0.001")
    print("5. Klik 'BLAST' dan tunggu hasilnya")

# Jalankan fungsi jika polyprotein_seq tersedia
if polyprotein_seq:
    print("Menerjemahkan sequence DNA menjadi protein...")
    translate_and_save_for_blast(polyprotein_seq)
else:
    print("Error: Sequence polyprotein tidak ditemukan!")

# Tampilkan beberapa karakter awal protein untuk verifikasi
if polyprotein_seq:
    protein = polyprotein_seq.translate()
    print("\nAwal sequence protein (20 asam amino pertama):")
    print(str(protein)[:20])

Menerjemahkan sequence DNA menjadi protein...
File protein telah disimpan sebagai protein_untuk_blast.fasta

Langkah BLAST:
1. Buka https://blast.ncbi.nlm.nih.gov/Blast.cgi
2. Pilih 'Protein BLAST (blastp)'
3. Upload file FASTA protein yang telah disimpan
4. Atur parameter BLAST:
   - Database: nr (non-redundant protein sequences)
   - Max target sequences: 10
   - Expect threshold: 0.001
5. Klik 'BLAST' dan tunggu hasilnya

Awal sequence protein (20 asam amino pertama):
MAKPVVKIDVEPESHFAKQV


In [ ]:
# 7. Pemodelan Struktur Menggunakan SWISS MODEL
# ---------------------------------

# Import library yang diperlukan
from Bio import SeqIO
from Bio.Seq import Seq
from Bio.SeqRecord import SeqRecord
import re

def clean_sequence_for_modeling(nucleotide_seq, output_file="clean_protein_for_modeling.fasta"):
    """
    Membersihkan dan mempersiapkan sequence protein untuk Swiss-Model
    """
    # Membersihkan sequence DNA dari karakter non-ATGC
    clean_dna = re.sub(r'[^ATGC]', '', str(nucleotide_seq))
    clean_dna = Seq(clean_dna)
    
    # Menerjemahkan ke protein
    protein_seq = clean_dna.translate()
    
    # Membersihkan sequence protein, hanya menerima karakter asam amino standar
    valid_aa = set("ACDEFGHIKLMNPQRSTVWY")
    clean_protein = ''.join(aa for aa in str(protein_seq) if aa in valid_aa)
    
    # Membuat SeqRecord dengan sequence yang sudah bersih
    record = SeqRecord(
        Seq(clean_protein),
        id="BFV_structural_polyprotein",
        description="Clean Barmah Forest Virus Structural Polyprotein"
    )
    
    # Menyimpan dalam format FASTA
    SeqIO.write(record, output_file, "fasta")
    
    print(f"File protein yang sudah dibersihkan telah disimpan sebagai {output_file}")
    print("\nPanduan penggunaan Swiss-Model:")
    print("1. Buka https://swissmodel.expasy.org/")
    print("2. Klik 'Start Modelling'")
    print("3. Upload file FASTA yang telah dibuat")
    print("4. Pilih 'Automated Mode'")
    print("5. Tunggu proses pemodelan selesai")
    
    return clean_protein

# Jalankan fungsi jika polyprotein_seq tersedia
if polyprotein_seq:
    print("Membersihkan dan mempersiapkan sequence untuk Swiss-Model...")
    clean_protein = clean_sequence_for_modeling(polyprotein_seq)
    
    print("\nInformasi sequence yang telah dibersihkan:")
    print(f"Panjang protein: {len(clean_protein)} asam amino")
    print("20 asam amino pertama:")
    print(clean_protein[:20])
    print("\nSequence siap untuk diupload ke Swiss-Model")
    
    # Menyimpan sequence dalam format yang mudah dicopy
    with open("protein_sequence.txt", "w") as f:
        f.write(f">BFV_structural_polyprotein\n{clean_protein}")
    print("\nSequence juga telah disimpan dalam 'protein_sequence.txt' untuk backup")
    
else:
    print("Error: Sequence polyprotein tidak ditemukan!")

Membersihkan dan mempersiapkan sequence untuk Swiss-Model...
File protein yang sudah dibersihkan telah disimpan sebagai clean_protein_for_modeling.fasta

Panduan penggunaan Swiss-Model:
1. Buka https://swissmodel.expasy.org/
2. Klik 'Start Modelling'
3. Upload file FASTA yang telah dibuat
4. Pilih 'Automated Mode'
5. Tunggu proses pemodelan selesai

Informasi sequence yang telah dibersihkan:
Panjang protein: 2296 asam amino
20 asam amino pertama:
MAKPVVKIDVEPESHFAKQV

Sequence siap untuk diupload ke Swiss-Model

Sequence juga telah disimpan dalam 'protein_sequence.txt' untuk backup
